# Introduction


The efficient scheduling of classes in Higher Education Institutions (HEIs) is crucial for ensuring optimal resource utilization and a seamless learning experience. However, manual scheduling processes are prone to errors and often fail to account for complex interdependencies between factors such as teacher availability, classroom capacities, and course requirements. To address these challenges, this project focuses on developing an automated scheduling system using advanced computational tools. By leveraging optimization techniques and prescriptive analysis, the system aims to generate conflict-free schedules that meet all established criteria, fostering a harmonized educational environment and enhancing the overall efficiency of HEI operations.

# Main Objectives

1. The main objective is to create an automated system for generating schedules in an HEI.<br>
2. The system should optimize the allocation of resources such as physical spaces and teaching schedules to meet the needs of students and faculty.<br>
3. It should aim to avoid conflicts and unwanted overlaps in schedules.<br>
4. The solution should employ optimization techniques and prescriptive analysis to ensure the harmonization and effectiveness of schedules.<br>

# Instalations

First I start instaling some libraries

<b>Instaled pyomo using the pip command</b>
pip install pyomo





conda install -c conda-forge glpk

# Definição do problema

Durante a elaboração deste problema defini algumas regras basicas, o horario de aulas sera compreendido entre as 09:00 da manhã e as 18:00 da tarde, de segunda a sexta-feira. Simulando assim um horario de aulas do IPCA.
Com estas constrições temos: 
Não pode haver mais que 8 horas de aulas por dia;
Os alunos tem que ter pelo menos uma manhã ou uma tarde livres;



In [8]:
import pyomo.environ as pyo
import pyomo.kernel as pmo
import numpy as np
import pandas as pd

# Define constants
WEEKENDS = ['Saturday', 'Sunday']
MAX_LESSONS_PER_DAY = 4
MAX_DAYS_PER_WEEK = 5

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
time_slot = ['Morning_09_11', 'Morning_11_13', 'Afternoon_14_16', 'Afternoon_16_18']

# class1 = ['Math_1', 'Math_2', 'Science1', 'English1', 'IT', 'IT_LAB']
# class2 = ['History', 'Literature', 'Art', 'Music', 'Drama', 'Creative_Writing']
# Define classes and number of students, class 1 have 30 students class 2 have 25
class1 = {'Math_1': 30, 'Math_2': 30, 'Science1': 30, 'English1': 30, 'IT': 30, 'IT_LAB': 30} 
class2 = {'History': 25, 'Literature': 25, 'Art': 25, 'Music': 25, 'Drama': 25, 'Creative_Writing': 25}

# Define number of students for each class
num_students = {**class1, **class2}

# Define available hours per week for each lesson
lesson_hours = {'Math_1': 8, 'Math_2': 8, 'Science1': 6, 'English1': 2,
                'IT': 8, 'IT_LAB': 4, 'History': 8, 'Literature': 8,
                'Art': 4, 'Music': 2, 'Drama': 6, 'Creative_Writing': 8}

# Define teachers and their lessons
teachers = ['John', 'Isabella', 'Brandon', 'Rafael', 'Lyla', 'Smith']
teacher_lessons = {
    'John': ['Math_1', 'Math_2'],
    'Isabella': ['History'],
    'Brandon': ['Science1', 'English1', 'Literature'],
    'Rafael': ['Art', 'Music', 'Drama'],
    'Lyla': ['IT', 'IT_LAB'],
    'Smith': ['IT_LAB']
}

teacher_unavailability = {
    'Isabella': ['Monday', 'Friday'],
    'John': ['Morning_09_11', 'Morning_11_13']
}

# Define classrooms and their capacities
classrooms = ['IT_Classroom', 'Art_Classroom', 'Classroom_1', 'Classroom_2', 'Classroom_3']
# the intention here is to force a classroom to never be used
classroom_capacity = {'IT_Classroom': 30, 'Art_Classroom': 25, 'Classroom_1': 35, 'Classroom_2': 22, 'Classroom_3': 40} 

# Define model
model = pyo.ConcreteModel()

# Sets
model.teachers = pyo.Set(initialize=teachers)
model.lessons = pyo.Set(initialize=list(class1.keys()) + list(class2.keys()))
model.days = pyo.Set(initialize=[day for day in days if day not in WEEKENDS]) # since we only have school in week days
model.time_slots = pyo.Set(initialize=time_slot)

# Update the model to include classrooms as a set
model.classrooms = pyo.Set(initialize=classrooms)

# Decision Variables: Classroom assignments
model.classroom_assignment = pyo.Var(model.lessons, model.classrooms, 
                                     model.days, model.time_slots, 
                                     domain=pyo.Binary)

# Decision Variables
model.schedule = pyo.Var(model.teachers, 
                         model.days, 
                         model.time_slots, 
                         model.lessons, 
                         domain=pyo.Binary)

# Constraint: IT and IT_LAB can only have lessons in the IT_Classroom
def it_classroom_constraint(model, lesson, classroom, day, time):
    if lesson in ['IT', 'IT_LAB']:
        if classroom != 'IT_Classroom':
            return model.classroom_assignment[lesson, classroom, day, time] == 0
    return pyo.Constraint.Skip

model.it_classroom_constraint = pyo.Constraint(model.lessons, model.classrooms, 
                                               model.days, model.time_slots, 
                                               rule=it_classroom_constraint)

# Constraint: Art can only have lessons in the Art_Classroom
def art_classroom_constraint(model, lesson, classroom, day, time):
    if lesson == 'Art':
        if classroom != 'Art_Classroom':
            return model.classroom_assignment[lesson, classroom, day, time] == 0
    return pyo.Constraint.Skip

model.art_classroom_constraint = pyo.Constraint(model.lessons, model.classrooms, 
                                                model.days, model.time_slots, 
                                                rule=art_classroom_constraint)

# Constraint: Ensure classroom capacity is not exceeded
def classroom_capacity_constraint_rule(model, classroom, day, time):
    total_students = sum(model.classroom_assignment[lesson, classroom, day, time] * num_students[lesson]
                         for lesson in model.lessons)
    return total_students <= classroom_capacity[classroom]

model.classroom_capacity_constraint = pyo.Constraint(model.classrooms, model.days, model.time_slots, 
                                                      rule=classroom_capacity_constraint_rule)




# Constraint: Total hours for each lesson should not exceed its availability
def availability_constraint(model, lesson):
    total_hours = sum(model.schedule[teacher, day, time, lesson] 
                      for teacher in model.teachers
                      for day in model.days
                      for time in model.time_slots)
    return total_hours <= lesson_hours[lesson]

model.availability_constraint = pyo.Constraint(model.lessons, rule=availability_constraint)

# Constraint: Isabella prefers not to work on Mondays and Fridays
def isabella_unavailability_constraint(model, teacher, day):
    if teacher == 'Isabella' and day in teacher_unavailability['Isabella']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for time in model.time_slots
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

model.isabella_unavailability_constraint = pyo.Constraint(model.teachers, model.days, rule=isabella_unavailability_constraint)

# Constraint: John can only work in the afternoon
def john_unavailability_constraint(model, teacher, time):
    if teacher == 'John' and time in teacher_unavailability['John']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for day in model.days
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

model.john_unavailability_constraint = pyo.Constraint(model.teachers, model.time_slots, rule=john_unavailability_constraint)


Constants and Data Definition:

You've defined constants such as weekends, maximum number of lessons per day, and maximum days per week.
You've also categorized classes into two groups (class1 and class2), representing different subjects or courses.
Teacher and Lesson Definition:

You've listed the teachers and assigned each teacher specific lessons they are responsible for teaching.
This allows you to consider teacher availability and subject expertise when generating the schedule.
Time and Day Definition:

You've defined days of the week and time slots, considering typical school scheduling constraints.
Model Initialization:

You've initialized a concrete model using Pyomo.
Defined sets for teachers, lessons, days, and time slots, which serve as the basis for decision variables and constraints.
Decision Variables:

You've declared decision variables named schedule to represent whether a teacher is assigned to teach a specific lesson at a particular time slot on a given day.
These variables are binary, indicating whether a particular teaching assignment is scheduled or not.

In [10]:
import pyomo.environ as pyo

# Define constants
WEEKENDS = ['Saturday', 'Sunday']
MAX_LESSONS_PER_DAY = 4
MAX_DAYS_PER_WEEK = 5

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
time_slot = ['Morning_09_11', 'Morning_11_13', 'Afternoon_14_16', 'Afternoon_16_18']

# Define classes and number of students, class 1 have 30 students class 2 have 25
class1 = {'Math_1': 30, 'Math_2': 30, 'Science1': 30, 'English1': 30, 'IT': 30, 'IT_LAB': 30} 
class2 = {'History': 25, 'Literature': 25, 'Art': 25, 'Music': 25, 'Drama': 25, 'Creative_Writing': 25}

# Define number of students for each class
num_students = {**class1, **class2}

# Define available hours per week for each lesson
lesson_hours = {'Math_1': 8, 'Math_2': 8, 'Science1': 6, 'English1': 2,
                'IT': 8, 'IT_LAB': 4, 'History': 8, 'Literature': 8,
                'Art': 4, 'Music': 2, 'Drama': 6, 'Creative_Writing': 8}

# Define teachers and their lessons
teachers = ['John', 'Isabella', 'Brandon', 'Rafael', 'Lyla', 'Smith']
teacher_lessons = {
    'John': ['Math_1', 'Math_2'],
    'Isabella': ['History'],
    'Brandon': ['Science1', 'English1', 'Literature'],
    'Rafael': ['Art', 'Music', 'Drama'],
    'Lyla': ['IT', 'IT_LAB'],
    'Smith': ['IT_LAB']
}

teacher_unavailability = {
    'Isabella': ['Monday', 'Friday'],
    'John': ['Morning_09_11', 'Morning_11_13']
}

# Define classrooms and their capacities
classrooms = ['IT_Classroom', 'Art_Classroom', 'Classroom_1', 'Classroom_2', 'Classroom_3']
# the intention here is to force a classroom to never be used
classroom_capacity = {'IT_Classroom': 30, 'Art_Classroom': 25, 'Classroom_1': 35, 'Classroom_2': 22, 'Classroom_3': 40} 

# Define model
model = pyo.ConcreteModel()

# Sets
model.teachers = pyo.Set(initialize=teachers)
model.lessons = pyo.Set(initialize=list(class1.keys()) + list(class2.keys()))
model.days = pyo.Set(initialize=[day for day in days if day not in WEEKENDS]) # since we only have school in week days
model.time_slots = pyo.Set(initialize=time_slot)

# Update the model to include classrooms as a set
model.classrooms = pyo.Set(initialize=classrooms)

# Decision Variables: Classroom assignments
model.classroom_assignment = pyo.Var(model.lessons, model.classrooms, 
                                     model.days, model.time_slots, 
                                     domain=pyo.Binary)

# Decision Variables
model.schedule = pyo.Var(model.teachers, 
                         model.days, 
                         model.time_slots, 
                         model.lessons, 
                         domain=pyo.Binary)

# Constraint: IT and IT_LAB can only have lessons in the IT_Classroom
def it_classroom_constraint(model, lesson, classroom, day, time):
    if lesson in ['IT', 'IT_LAB']:
        if classroom != 'IT_Classroom':
            return model.classroom_assignment[lesson, classroom, day, time] == 0
    return pyo.Constraint.Skip

model.it_classroom_constraint = pyo.Constraint(model.lessons, model.classrooms, 
                                               model.days, model.time_slots, 
                                               rule=it_classroom_constraint)

# Constraint: Art can only have lessons in the Art_Classroom
def art_classroom_constraint(model, lesson, classroom, day, time):
    if lesson == 'Art':
        if classroom != 'Art_Classroom':
            return model.classroom_assignment[lesson, classroom, day, time] == 0
    return pyo.Constraint.Skip

model.art_classroom_constraint = pyo.Constraint(model.lessons, model.classrooms, 
                                                model.days, model.time_slots, 
                                                rule=art_classroom_constraint)

# Constraint: Ensure classroom capacity is not exceeded
def classroom_capacity_constraint_rule(model, classroom, day, time):
    total_students = sum(model.classroom_assignment[lesson, classroom, day, time] * num_students[lesson]
                         for lesson in model.lessons)
    return total_students <= classroom_capacity[classroom]

model.classroom_capacity_constraint = pyo.Constraint(model.classrooms, model.days, model.time_slots, 
                                                      rule=classroom_capacity_constraint_rule)

# Constraint: Total hours for each lesson should not exceed its availability
def availability_constraint(model, lesson):
    total_hours = sum(model.schedule[teacher, day, time, lesson] 
                      for teacher in model.teachers
                      for day in model.days
                      for time in model.time_slots)
    return total_hours <= lesson_hours[lesson]

model.availability_constraint = pyo.Constraint(model.lessons, rule=availability_constraint)

# Constraint: Isabella prefers not to work on Mondays and Fridays
def isabella_unavailability_constraint(model, teacher, day):
    if teacher == 'Isabella' and day in teacher_unavailability['Isabella']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for time in model.time_slots
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

model.isabella_unavailability_constraint = pyo.Constraint(model.teachers, model.days, rule=isabella_unavailability_constraint)

# Constraint: John can only work in the afternoon
def john_unavailability_constraint(model, teacher, time):
    if teacher == 'John' and time in teacher_unavailability['John']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for day in model.days
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

model.john_unavailability_constraint = pyo.Constraint(model.teachers, model.time_slots, rule=john_unavailability_constraint)

# Define objective function (if any)

# Example: Minimize the total number of scheduled hours across all teachers, days, and lessons
def minimize_total_hours(model):
    return sum(model.schedule[teacher, day, time, lesson]
               for teacher in model.teachers
               for day in model.days
               for time in model.time_slots
               for lesson in model.lessons)

model.objective = pyo.Objective(rule=minimize_total_hours, sense=pyo.minimize)

# Alternatively, you can define other objectives based on the specific goals of your scheduling problem.

# Solve the model
solver = pyo.SolverFactory('glpk', executable=r'C:\ProgramData\anaconda3\Library\bin\glpsol.exe')
results = solver.solve(model)

# Check the solver termination condition
if str(results.solver.termination_condition) == "optimal":
    print("Solver terminated with optimal solution.")
else:
    print("Solver terminated with non-optimal solution.")

#Print the schedule (if needed)
for teacher in model.teachers:
    for day in model.days:
        for time in model.time_slots:
            for lesson in model.lessons:
                if model.schedule[teacher, day, time, lesson].value == 1:
                    print(f"Teacher {teacher} teaches {lesson} on {day} during {time}.")

#Additional post-processing or analysis can be done based on the optimization results.


Solver terminated with optimal solution.


In [15]:
# Solve the model
solver = pyo.SolverFactory('glpk', executable=r'C:\ProgramData\anaconda3\Library\bin\glpsol.exe')
results = solver.solve(model)

# Check the solver termination condition
if str(results.solver.termination_condition) == "optimal":
    print("Solver terminated with optimal solution.")
else:
    print("Solver terminated with non-optimal solution.")

# Check if the schedule variables are assigned values
print("Checking if schedule variables are assigned:")
for teacher in model.teachers:
    for day in model.days:
        for time in model.time_slots:
            for lesson in model.lessons:
                if model.schedule[teacher, day, time, lesson].value is not None:
                    print(f"Schedule variable for Teacher {teacher}, Lesson {lesson}, Day {day}, Time {time}: {model.schedule[teacher, day, time, lesson].value}")


Solver terminated with optimal solution.
Checking if schedule variables are assigned:
Schedule variable for Teacher John, Lesson Math_1, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson Math_2, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson Science1, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson English1, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson IT, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson IT_LAB, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson History, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson Literature, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson Art, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson Music, Day Monday, Time Morning_09_11: 0.0
Schedule variable for Teacher John, Lesson 

In [19]:
import pyomo.environ as pyo

# Define constants
WEEKENDS = ['Saturday', 'Sunday']
MAX_LESSONS_PER_DAY = 4
MAX_DAYS_PER_WEEK = 5

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
time_slot = ['Morning_09_11', 'Morning_11_13', 'Afternoon_14_16', 'Afternoon_16_18']

# Define classes and number of students, class 1 have 30 students class 2 have 25
class1 = {'Math_1': 30, 'Math_2': 30, 'Science1': 30, 'English1': 30, 'IT': 30, 'IT_LAB': 30} 
class2 = {'History': 25, 'Literature': 25, 'Art': 25, 'Music': 25, 'Drama': 25, 'Creative_Writing': 25}

# Define number of students for each class
num_students = {**class1, **class2}

# Define available hours per week for each lesson
lesson_hours = {'Math_1': 8, 'Math_2': 8, 'Science1': 6, 'English1': 2,
                'IT': 8, 'IT_LAB': 4, 'History': 8, 'Literature': 8,
                'Art': 4, 'Music': 2, 'Drama': 6, 'Creative_Writing': 8}

# Define teachers and their lessons
teachers = ['John', 'Isabella', 'Brandon', 'Rafael', 'Lyla', 'Smith']
teacher_lessons = {
    'John': ['Math_1', 'Math_2'],
    'Isabella': ['History'],
    'Brandon': ['Science1', 'English1', 'Literature'],
    'Rafael': ['Art', 'Music', 'Drama'],
    'Lyla': ['IT', 'IT_LAB'],
    'Smith': ['IT_LAB']
}

teacher_unavailability = {
    'Isabella': ['Monday', 'Friday'],
    'John': ['Morning_09_11', 'Morning_11_13']
}

# Define classrooms and their capacities
classrooms = ['IT_Classroom', 'Art_Classroom', 'Classroom_1', 'Classroom_2', 'Classroom_3']
# the intention here is to force a classroom to never be used
classroom_capacity = {'IT_Classroom': 30, 'Art_Classroom': 25, 'Classroom_1': 35, 'Classroom_2': 22, 'Classroom_3': 40} 


# Define model
model = pyo.ConcreteModel()

# Sets
model.teachers = pyo.Set(initialize=teachers)
model.lessons = pyo.Set(initialize=list(class1.keys()) + list(class2.keys()))
model.days = pyo.Set(initialize=[day for day in days if day not in WEEKENDS])
model.time_slots = pyo.Set(initialize=time_slot)
model.classrooms = pyo.Set(initialize=classrooms)

# Decision Variables
model.schedule = pyo.Var(model.teachers, model.days, model.time_slots, model.lessons, domain=pyo.Binary)
model.classroom_assignment = pyo.Var(model.lessons, model.classrooms, model.days, model.time_slots, domain=pyo.Binary)

# Constraint: IT and IT_LAB can only have lessons in the IT_Classroom
def it_classroom_constraint(model, lesson, classroom, day, time):
    if lesson in ['IT', 'IT_LAB']:
        if classroom != 'IT_Classroom':
            return model.classroom_assignment[lesson, classroom, day, time] == 0
    return pyo.Constraint.Skip

model.it_classroom_constraint = pyo.Constraint(model.lessons, model.classrooms, model.days, model.time_slots, rule=it_classroom_constraint)

# Constraint: Art can only have lessons in the Art_Classroom
def art_classroom_constraint(model, lesson, classroom, day, time):
    if lesson == 'Art':
        if classroom != 'Art_Classroom':
            return model.classroom_assignment[lesson, classroom, day, time] == 0
    return pyo.Constraint.Skip

model.art_classroom_constraint = pyo.Constraint(model.lessons, model.classrooms, model.days, model.time_slots, rule=art_classroom_constraint)

# Constraint: Ensure classroom capacity is not exceeded
def classroom_capacity_constraint_rule(model, classroom, day, time):
    total_students = sum(model.classroom_assignment[lesson, classroom, day, time] * num_students[lesson]
                         for lesson in model.lessons)
    return total_students <= classroom_capacity[classroom]

model.classroom_capacity_constraint = pyo.Constraint(model.classrooms, model.days, model.time_slots, 
                                                      rule=classroom_capacity_constraint_rule)

# Constraint: Total hours for each lesson should not exceed its availability
def availability_constraint(model, lesson):
    total_hours = sum(model.schedule[teacher, day, time, lesson] 
                      for teacher in model.teachers
                      for day in model.days
                      for time in model.time_slots)
    return total_hours <= lesson_hours[lesson]

model.availability_constraint = pyo.Constraint(model.lessons, rule=availability_constraint)

# Constraint: Isabella prefers not to work on Mondays and Fridays
def isabella_unavailability_constraint(model, teacher, day):
    if teacher == 'Isabella' and day in teacher_unavailability['Isabella']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for time in model.time_slots
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

model.isabella_unavailability_constraint = pyo.Constraint(model.teachers, model.days, rule=isabella_unavailability_constraint)

# Constraint: John can only work in the afternoon
def john_unavailability_constraint(model, teacher, time):
    if teacher == 'John' and time in teacher_unavailability['John']:
        return sum(model.schedule[teacher, day, time, lesson] 
                   for day in model.days
                   for lesson in model.lessons) == 0
    else:
        return pyo.Constraint.Skip

model.john_unavailability_constraint = pyo.Constraint(model.teachers, model.time_slots, rule=john_unavailability_constraint)

# Objective function: Minimize conflicts and maximize utilization (could be extended)
model.obj = pyo.Objective(expr=0, sense=pyo.minimize)

# Solve the model
solver = pyo.SolverFactory('glpk', executable=r'C:\ProgramData\anaconda3\Library\bin\glpsol.exe')
solver.solve(model)
results = solver.solve(model)

# Check if the solver successfully solved the model
if results.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("Solver terminated successfully. Printing the schedule:")
    
    # Print teacher schedule
    print("Teacher Schedule:")
    for teacher in model.teachers:
        for day in model.days:
            for time_slot in model.time_slots:
                for lesson in model.lessons:
                    if model.schedule[teacher, day, time_slot, lesson].value == 1:
                        print(f"Teacher {teacher} is teaching {lesson} on {day} at {time_slot}")

    # Print classroom assignments
    print("\nClassroom Assignments:")
    for lesson in model.lessons:
        for classroom in model.classrooms:
            for day in model.days:
                for time_slot in model.time_slots:
                    if model.classroom_assignment[lesson, classroom, day, time_slot].value == 1:
                        print(f"{lesson} is assigned to {classroom} on {day} at {time_slot}")

else:
    print("Solver terminated with non-optimal solution.")

Solver terminated successfully. Printing the schedule:
